In [1]:
import os
import numpy as np
import tensorflow as tf 
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16 
from tensorflow.keras import layers, models 
from tensorflow.keras.optimizers import Adam  

In [2]:
# Directories
fake_dir = "D:\\Revature\\P1\\temp\\training_fake"
real_dir = "D:\\Revature\\P1\\temp\\training_real" 

In [3]:
# Function to load images from a directory and assign a label
def load_images_from_folder(folder, label):
    images = [] 
    labels = []
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename) 
        img = tf.keras.preprocessing.image.load_img(img_path, target_size=(128, 128))
        img_array = tf.keras.preprocessing.image.img_to_array(img)
        images.append(img_array) 
        labels.append(label)
    return np.array(images), np.array(labels)

In [4]:
# Load fake and real images and their corresponding labels
fake_images, fake_labels = load_images_from_folder(fake_dir, label=0) 
real_images, real_labels = load_images_from_folder(real_dir, label=1) 

In [5]:
# Split fake images into training, validation, and test sets
X_train_fake, X_test_fake, y_train_fake, y_test_fake = train_test_split(fake_images, fake_labels, test_size=0.2, random_state=42)
X_train_fake, X_val_fake, y_train_fake, y_val_fake = train_test_split(X_train_fake, y_train_fake, test_size=0.2, random_state=42)

In [6]:
# Split real images into training, validation, and test sets
X_train_real, X_test_real, y_train_real, y_test_real = train_test_split(real_images, real_labels, test_size=0.2, random_state=42)
X_train_real, X_val_real, y_train_real, y_val_real = train_test_split(X_train_real, y_train_real, test_size=0.2, random_state=42)

In [7]:
# Combine the training sets
X_train = np.concatenate((X_train_fake, X_train_real), axis=0)
y_train = np.concatenate((y_train_fake, y_train_real), axis=0)


In [8]:
# Combine the validation sets
X_val = np.concatenate((X_val_fake, X_val_real), axis=0) 
y_val = np.concatenate((y_val_fake, y_val_real), axis=0)

In [9]:
# Combine the test sets
X_test = np.concatenate((X_test_fake, X_test_real), axis=0)
y_test = np.concatenate((y_test_fake, y_test_real), axis=0)

In [10]:
# Shuffle the combined training set
train_indices = np.arange(X_train.shape[0])
np.random.shuffle(train_indices)
X_train = X_train[train_indices] 
y_train = y_train[train_indices]  

In [11]:
# Shuffle the combined validation set
val_indices = np.arange(X_val.shape[0])
np.random.shuffle(val_indices)
X_val = X_val[val_indices] 
y_val = y_val[val_indices]

In [12]:
# Shuffle the combined test set
test_indices = np.arange(X_test.shape[0])
np.random.shuffle(test_indices)
X_test = X_test[test_indices]
y_test = y_test[test_indices]

In [13]:
# Print dataset sizes
print(f"Training set: {X_train.shape[0]} images")
print(f"Validation set: {X_val.shape[0]} images") 
print(f"Testing set: {X_test.shape[0]} images")

Training set: 1151 images
Validation set: 289 images
Testing set: 361 images


In [14]:
# Create ImageDataGenerator instances
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True, 
    fill_mode='nearest' 
)

In [15]:
val_datagen = ImageDataGenerator(rescale=1.0/255)

In [16]:
# Load the VGG16 model with pre-trained weights, excluding the top (fully connected) layers
vgg16_base = VGG16(weights='imagenet', include_top=False, input_shape=(128, 128, 3)) 

In [17]:
# Freeze the base layers so they are not trained during fine-tuning
vgg16_base.trainable = False 

In [18]:
# Create a new model
model = models.Sequential() 

In [19]:
# Add the VGG16 base model
model.add(vgg16_base)

In [20]:
# Add custom layers
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.5))  # Regularization to prevent overfitting
model.add(layers.Dense(1, activation='sigmoid'))  # Output layer for binary classification 

In [21]:
model.compile(optimizer=Adam(learning_rate=1e-4), loss='binary_crossentropy', metrics=['accuracy'])

In [22]:
# Fit the model
history = model.fit( 
    train_datagen.flow(X_train, y_train, batch_size=32),
    steps_per_epoch=len(X_train) // 32,
    validation_data=val_datagen.flow(X_val, y_val, batch_size=32),
    epochs=30
) 

Epoch 1/30
35/35 [==============================] - 50s 1s/step - loss: 0.7395 - accuracy: 0.5576 - val_loss: 0.6599 - val_accuracy: 0.6263
Epoch 2/30
35/35 [==============================] - 52s 1s/step - loss: 0.6766 - accuracy: 0.5862 - val_loss: 0.6763 - val_accuracy: 0.5709
Epoch 3/30
35/35 [==============================] - 52s 1s/step - loss: 0.6611 - accuracy: 0.6095 - val_loss: 0.6414 - val_accuracy: 0.6263
Epoch 4/30
35/35 [==============================] - 61s 2s/step - loss: 0.6466 - accuracy: 0.6152 - val_loss: 0.6445 - val_accuracy: 0.6055
Epoch 5/30
35/35 [==============================] - 65s 2s/step - loss: 0.6512 - accuracy: 0.6139 - val_loss: 0.6281 - val_accuracy: 0.6332
Epoch 6/30
35/35 [==============================] - 56s 2s/step - loss: 0.6289 - accuracy: 0.6425 - val_loss: 0.6263 - val_accuracy: 0.6263
Epoch 7/30
35/35 [==============================] - 58s 2s/step - loss: 0.6347 - accuracy: 0.6300 - val_loss: 0.6463 - val_accuracy: 0.6298
Epoch 8/30
35/35 [==

In [24]:
# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(val_datagen.flow(X_test, y_test, batch_size=32))
print(f"Test Accuracy: {test_acc:.4f}")


12/12 [==============================] - 13s 1s/step - loss: 0.6575 - accuracy: 0.6122
Test Accuracy: 0.6122


In [25]:
# Unfreeze the last few layers of VGG16
vgg16_base.trainable = True
for layer in vgg16_base.layers[:-4]:  # Unfreeze the last 4 layers
    layer.trainable = False

In [26]:
# Re-compile the model with a lower learning rate
model.compile(optimizer=Adam(learning_rate=1e-5), loss='binary_crossentropy', metrics=['accuracy'])

In [27]:
# Continue training 
history_fine = model.fit( 
    train_datagen.flow(X_train, y_train, batch_size=32),
    steps_per_epoch=len(X_train) ,
    validation_data=val_datagen.flow(X_val, y_val, batch_size=32),
    epochs=10
)

Epoch 1/10
35/35 [==============================] - 67s 2s/step - loss: 0.5673 - accuracy: 0.6872 - val_loss: 0.5973 - val_accuracy: 0.6471
Epoch 2/10
35/35 [==============================] - 67s 2s/step - loss: 0.5562 - accuracy: 0.7122 - val_loss: 0.6243 - val_accuracy: 0.6263
Epoch 3/10
35/35 [==============================] - 67s 2s/step - loss: 0.5383 - accuracy: 0.7373 - val_loss: 0.5943 - val_accuracy: 0.6990
Epoch 4/10
35/35 [==============================] - 66s 2s/step - loss: 0.5376 - accuracy: 0.7283 - val_loss: 0.6036 - val_accuracy: 0.6747
Epoch 5/10
35/35 [==============================] - 69s 2s/step - loss: 0.5384 - accuracy: 0.7122 - val_loss: 0.6196 - val_accuracy: 0.6644
Epoch 6/10
35/35 [==============================] - 67s 2s/step - loss: 0.5350 - accuracy: 0.7310 - val_loss: 0.6019 - val_accuracy: 0.6609
Epoch 7/10
35/35 [==============================] - 68s 2s/step - loss: 0.5032 - accuracy: 0.7534 - val_loss: 0.6201 - val_accuracy: 0.6609
Epoch 8/10
35/35 [==

In [28]:
# Evaluate again
test_loss, test_acc = model.evaluate(val_datagen.flow(X_test, y_test, batch_size=32))
print(f"Test Accuracy after fine-tuning: {test_acc:.4f}")

12/12 [==============================] - 13s 1s/step - loss: 0.7322 - accuracy: 0.6039
Test Accuracy after fine-tuning: 0.6039


In [29]:
# Save the model
model.save("D:\\Revature\\P1\\models\\my_vgg16_model.h5")

In [ ]:
# Define the path to the image you want to predict
image_path = r"D:\Revature\P1\28-8-24\training_fake\easy_38_0010.jpg"
